In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


X = tf.placeholder(tf.float32, [None, 784])
y_true = tf.placeholder(tf.float32, [None, 10])

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


__把加layer寫成一個function__

這編寫的add_layer其實只是幫我做了把input資料，乘上一個W矩陣加上bias，也就是一層fully connected層的意思，最後activation function再經過一個softmax，最後輸出就是網路的輸出，已經可以和ground truth label算cross entropy

__NOTE: __裡面的 __W__ 和 __bias__ 宣告的方式會影響其初始化的方式，往往對訓練品質會有很大的影響，需要作好的調整

In [2]:
def add_layer(inputs, in_size, out_size, activation_function=None):
    """
    args1 input: 輸入input的placeholder
    args2 in_size: input維度
    args3 out_size: output維度
    args4 activation_function: 激勵函數，通常是tf.nn下的函式
    """
    W = tf.Variable(tf.zeros([in_size, out_size]))
    bias = tf.Variable(tf.zeros([out_size]) + 0.1)
    WX_plus_b = tf.matmul(inputs, W) + bias
    if activation_function is None:
        outputs = WX_plus_b
    else:
        outputs = activation_function(WX_plus_b)
    return outputs

__ML第一步-定義模型__

這邊使用寫好的add_layer函式，使用softmax函式，輸入維度維784，輸出維度為10

In [3]:
# hidden_layer_1_output = add_layer(X, 784, 100, activation_function=tf.nn.relu)
# output = add_layer(hidden_layer_1_output, 100, 10, activation_function=tf.nn.softmax)

In [4]:
output = add_layer(X, 784, 10, activation_function=tf.nn.softmax)

ML第二步-定義Loss Function

In [5]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_true * tf.log(output), reduction_indices=[1]))

ML第三步-Gradient Descent

In [6]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

--------

開始執行computational graph

In [21]:
init = tf.global_variables_initializer()

In [22]:
sess = tf.Session()

In [23]:
sess.run(init)

In [27]:
from time import time

start = time()

for i in range(2000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
#     sess.run(train_step, feed_dict={X: batch_xs, y_true: batch_ys})
    """
    如果執行sess.run的時候，也一起執行cross_entropy這個node
    則sess.run可以回傳「當次iteration的loss」，如以下程式碼
    """
    _, current_loss = sess.run([train_step, cross_entropy], feed_dict={X: batch_xs, y_true: batch_ys})

    if i % 50 == 0:
        print('第', i, '輪訓練的Loss為', current_loss)
        
end = time()
print('訓練花費', (end-start), '秒')

第 0 輪訓練的Loss為 0.184176
第 50 輪訓練的Loss為 0.284074
第 100 輪訓練的Loss為 0.268324
第 150 輪訓練的Loss為 0.212792
第 200 輪訓練的Loss為 0.327426
第 250 輪訓練的Loss為 0.334656
第 300 輪訓練的Loss為 0.182557
第 350 輪訓練的Loss為 0.179001
第 400 輪訓練的Loss為 0.39459
第 450 輪訓練的Loss為 0.205549
第 500 輪訓練的Loss為 0.31461
第 550 輪訓練的Loss為 0.3483
第 600 輪訓練的Loss為 0.23946
第 650 輪訓練的Loss為 0.288157
第 700 輪訓練的Loss為 0.259424
第 750 輪訓練的Loss為 0.229819
第 800 輪訓練的Loss為 0.182715
第 850 輪訓練的Loss為 0.27714
第 900 輪訓練的Loss為 0.231333
第 950 輪訓練的Loss為 0.340513
第 1000 輪訓練的Loss為 0.181208
第 1050 輪訓練的Loss為 0.346117
第 1100 輪訓練的Loss為 0.364409
第 1150 輪訓練的Loss為 0.349409
第 1200 輪訓練的Loss為 0.210036
第 1250 輪訓練的Loss為 0.264438
第 1300 輪訓練的Loss為 0.52017
第 1350 輪訓練的Loss為 0.287668
第 1400 輪訓練的Loss為 0.251905
第 1450 輪訓練的Loss為 0.409638
第 1500 輪訓練的Loss為 0.243659
第 1550 輪訓練的Loss為 0.293755
第 1600 輪訓練的Loss為 0.31353
第 1650 輪訓練的Loss為 0.365671
第 1700 輪訓練的Loss為 0.192871
第 1750 輪訓練的Loss為 0.106423
第 1800 輪訓練的Loss為 0.219986
第 1850 輪訓練的Loss為 0.284489
第 1900 輪訓練的Loss為 0.265077
第 1950 輪訓練的Loss為 

---

evaluation

In [28]:
correct_prediction = tf.equal(tf.argmax(output, 1), tf.argmax(y_true, 1)) # 注意這邊的 y 和 y_ 都是placeholder
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Testing Data Accuracy: ', sess.run(accuracy, feed_dict = {X: mnist.test.images, y_true: mnist.test.labels}))
print('Training Data Accuracy: ', sess.run(accuracy, feed_dict = {X: mnist.train.images, y_true: mnist.train.labels}))

Testing Data Accuracy:  0.9199
Training Data Accuracy:  0.924455
